# Predicting ESG Categories and Polarities

In [1]:
from ekorpkit import eKonf

if eKonf.is_colab():
    eKonf.mount_google_drive()
ws = eKonf.set_workspace(
    workspace="/workspace", 
    project="ekorpkit-book/exmaples", 
    task="esg", 
    log_level="INFO"
)
print("version:", ws.version)
print("project_dir:", ws.project_dir)

INFO:ekorpkit.base:Set environment variable EKORPKIT_DATA_ROOT=/workspace/data
INFO:ekorpkit.base:Set environment variable CACHED_PATH_CACHE_ROOT=/workspace/.cache/cached_path


version: 0.1.40.post0.dev57
project_dir: /workspace/projects/ekorpkit-book/exmaples
time: 944 ms (started: 2022-12-16 00:50:17 +00:00)


In [2]:
data_dir = "../data/econ_news_kr"


time: 282 µs (started: 2022-12-09 08:28:36 +00:00)


## Load data

In [3]:
filtered_data = eKonf.load_data("econ_news_filtered_20220905.parquet", data_dir)
filtered_data

,id,chunk_id,text,filename
0,11,0,문재인 대통령이 지난해 전용차로 선택한 수소전기차 '넥쏘'가 중·장년층 남성들로부터...,02100101.20201101113442001.txt
1,11,1,넥쏘는 국내 뿐만 아니라 해외에서도 코로나19 확산 여파를 이겨내고 질주하고 있다....,02100101.20201101113442001.txt
2,19,0,LG전자는 국내 최대 쇼핑 행사 ‘2020 코리아세일페스타’에 맞춰 11월 1일부터...,02100101.20201101125903001.txt
3,20,0,SK는 그룹 8개 관계사가 한국 최초로 ‘RE100’에 가입한다고 1일 밝혔다.\n...,02100101.20201101130118001.txt
4,20,1,SK 8개사가 신청서를 제출하면 더 클라이밋 그룹의 검토를 거친 후 가입이 최종 확...,02100101.20201101130118001.txt
...,...,...,...,...
160582,1090535,1,"업종별로 하락한 업종은 운수창고(-1.22%), 운수장비(-1.12%), 전기전자(...",02100851.20220228093428001.txt
160583,1090566,0,[회사채 미매각 현황]\n회사채 시장에 미매각이 속출하고 있다. 현재 연 1.25%...,02100851.20220228103653001.txt
160584,1090566,1,하지만 가뜩이나 악화된 회사채 시장 분위기에 사고 소식까지 전해지면서 아무도 수요예...,02100851.20220228103653001.txt
160585,1090566,2,이에 채권시장에서 자금을 운용하는 각 증권사들의 부담이 커지고 있다. 대부분이 총액...,02100851.20220228103653001.txt


time: 2.01 s (started: 2022-12-09 08:28:37 +00:00)


In [4]:
filtered_data.text[222]

'특히 반도체산업 회복 기대감이 높다. 삼성전자와 SK하이닉스는 코스피 시가총액 25%를 차지한다. 삼성전자는 내년 영업이익이 46조원대로 올해 대비 25% 이상 증가할 것이라는 전망이 나온다. 같은 기간 SK하이닉스 영업이익은 8조3000억원대로 70% 넘게 늘어날 듯 보인다.\n김용구 삼성증권 애널리스트는 “내년 국내 증시는 역사적 신고가 돌파에 나서는 대세 상승장 시작점에 해당된다”고 밝혔다. 근거는 ▲수출 실적 정상화 ▲전 세계적인 경기 부양 정책 환경 ▲외국인과 개인 수급 선순환 구도 본격화다.\nKB증권이 내세운 희망 키워드는 생산과 투자 회복이다. 선진국 소비는 코로나 팬데믹 이전을 넘어섰으나 생산은 아직 절반밖에 회복하지 못했기 때문에 내년 생산 증가를 기대해볼 만하다는 분석이다. 신동준 KB증권 리서치센터장은 “한국 기업이익은 올해 상반기 줄어들고 하반기 늘어나는 양상을 보였다”며 “내년 성장폭은 작아도 이익 정상화는 충분히 가능하다”고 봤다.\n신지윤 KTB투자증권 리서치센터장은 “저금리, 재정확장 정책, 원화 강세 현상은 증시로 자금을 끌어모으는 요인”이라며 “삼성전자와 현대차 등 대형주 실적이 좋아 주가 상승을 기대해볼 만하다”고 말했다. 명쾌하게 결론은 안 났지만 美 대선 불확실성을 벗어났다는 점도 호재로 평가된다.'

In [5]:
filtered_data[filtered_data.filename == "02100201.20200323170208001.txt"]

,id,chunk_id,text,filename
58469,385724,0,[머니투데이 박소연 기자] [당장 유동성보다는 전 세계 수요·구매력 감소가 더 큰 ...,02100201.20200323170208001.txt
58470,385724,1,"국내 전자업계는 공장이 한국과 중국, 베트남 지역에 있어 북미나 유럽발 셧다운으로 ...",02100201.20200323170208001.txt


## Predicting polarities

In [10]:
overrides=[
    '+model/transformer=classification',
    '+model/transformer/pretrained=ekonelectra-base',
]
model_cfg = eKonf.compose('model/transformer=classification', overrides)
model_cfg.name = "esg_polarity"
model_cfg.verbose = False
model_cfg.config.num_train_epochs = 2
model_cfg.config.max_seq_length = 256
model_cfg.config.train_batch_size = 32
model_cfg.config.eval_batch_size = 32
# set to your model dir
model_cfg.config.best_model_dir = '/workspace/.tmp/esg/polarity/best_model'

model = eKonf.instantiate(model_cfg)

INFO:ekorpkit.base:No method defined to call


In [9]:
model.predict(filtered_data.tail(5))

  0%|          | 0/5 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/1 [00:00<?, ?it/s]

,id,chunk_id,text,filename,pred_labels,raw_preds,pred_probs
160582,1090535,1,"업종별로 하락한 업종은 운수창고(-1.22%), 운수장비(-1.12%), 전기전자(...",02100851.20220228093428001.txt,Neutral,"{'Negative': 0.021272389449621617, 'Neutral': ...",0.558927
160583,1090566,0,[회사채 미매각 현황]\n회사채 시장에 미매각이 속출하고 있다. 현재 연 1.25%...,02100851.20220228103653001.txt,Negative,"{'Negative': 0.3028968002392749, 'Neutral': 0....",0.302897
160584,1090566,1,하지만 가뜩이나 악화된 회사채 시장 분위기에 사고 소식까지 전해지면서 아무도 수요예...,02100851.20220228103653001.txt,Negative,"{'Negative': 0.21217251856446653, 'Neutral': 0...",0.227077
160585,1090566,2,이에 채권시장에서 자금을 운용하는 각 증권사들의 부담이 커지고 있다. 대부분이 총액...,02100851.20220228103653001.txt,Neutral,"{'Negative': 0.16820714620311472, 'Neutral': 0...",0.807850
160586,1090615,0,[사진=아주경제 DB]\n우크라이나와 러시아가 협상 테이블을 마련한다는 소식에 국내...,02100851.20220228154344001.txt,Neutral,"{'Negative': 0.001653071269117536, 'Neutral': ...",0.287804


In [8]:
cfg = eKonf.compose(config_group='pipeline')
cfg.name = 'esg_polarities_20220905'
cfg.data_dir = data_dir
cfg.data_file = 'econ_news_filtered_20220905.parquet'
cfg._pipeline_ = ['predict']
cfg.predict.model = model_cfg
cfg.predict.output_dir = data_dir
cfg.predict.output_file = f'{cfg.name}-preds.parquet'
cfg.num_workers = 1
preds_df = eKonf.instantiate(cfg)
preds_df.head()

INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env
INFO:ekorpkit.base:setting environment variable CACHED_PATH_CACHE_ROOT to /workspace/.cache/cached_path
INFO:ekorpkit.base:setting environment variable KMP_DUPLICATE_LIB_OK to TRUE
INFO:ekorpkit.base:Applying pipe: functools.partial(<function predict at 0x7faa65df1ee0>)
INFO:ekorpkit.base:No method defined to call


  0%|          | 0/160587 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/10480 [00:00<?, ?it/s]

,id,chunk_id,text,filename,pred_labels,raw_preds,pred_probs
0,11,0,문재인 대통령이 지난해 전용차로 선택한 수소전기차 '넥쏘'가 중·장년층 남성들로부터...,02100101.20201101113442001.txt,Positive,"{'Negative': 0.003234388916771163, 'Neutral': ...",0.320464
1,11,1,넥쏘는 국내 뿐만 아니라 해외에서도 코로나19 확산 여파를 이겨내고 질주하고 있다....,02100101.20201101113442001.txt,Positive,"{'Negative': 0.005179156956969719, 'Neutral': ...",0.463171
2,19,0,LG전자는 국내 최대 쇼핑 행사 ‘2020 코리아세일페스타’에 맞춰 11월 1일부터...,02100101.20201101125903001.txt,Positive,"{'Negative': 0.0022622254938860858, 'Neutral':...",0.293573
3,20,0,SK는 그룹 8개 관계사가 한국 최초로 ‘RE100’에 가입한다고 1일 밝혔다.\n...,02100101.20201101130118001.txt,Positive,"{'Negative': 0.003873849489889034, 'Neutral': ...",0.457221
4,20,1,SK 8개사가 신청서를 제출하면 더 클라이밋 그룹의 검토를 거친 후 가입이 최종 확...,02100101.20201101130118001.txt,Positive,"{'Negative': 0.0033662562749769104, 'Neutral':...",0.387810


## Predicting categories

In [9]:
overrides=[
    '+model/transformer=classification',
    '+model/transformer/pretrained=ekonelectra-base',
]
model_cfg = eKonf.compose('model/transformer=classification', overrides)
model_cfg.name = "esg_topics"
model_cfg.verbose = False
model_cfg.config.num_train_epochs = 2
model_cfg.config.max_seq_length = 256
model_cfg.config.train_batch_size = 32
model_cfg.config.eval_batch_size = 32
# set to your model dir
model_cfg.config.best_model_dir = '/workspace/.tmp/esg/topic/best_model'

In [9]:
cfg = eKonf.compose(config_group='pipeline')
cfg.name = 'esg_categories_20220905'
cfg.data_dir = data_dir
cfg.data_file = 'econ_news_filtered.parquet'
cfg._pipeline_ = ['predict']
cfg.predict.model = model_cfg
cfg.predict.output_dir = data_dir
cfg.predict.output_file = f'{cfg.name}-preds.parquet'
cfg.num_workers = 1
cat_preds_df = eKonf.instantiate(cfg)
cat_preds_df.head()

INFO:ekorpkit.base:Applying pipe: functools.partial(<function predict at 0x7f1029331f70>)
INFO:ekorpkit.base:No method defined to call


  0%|          | 0/160587 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/10480 [00:00<?, ?it/s]

,id,chunk_id,text,filename,pred_labels,raw_preds,pred_probs
0,11,0,문재인 대통령이 지난해 전용차로 선택한 수소전기차 '넥쏘'가 중·장년층 남성들로부터...,02100101.20201101113442001.txt,E-환경혁신,"{'S-기업(공급망)동반성장/상생': 0.0035235610749915036, 'G...",0.274359
1,11,1,넥쏘는 국내 뿐만 아니라 해외에서도 코로나19 확산 여파를 이겨내고 질주하고 있다....,02100101.20201101113442001.txt,E-환경혁신,"{'S-기업(공급망)동반성장/상생': 0.00476169941298804, 'G-지...",0.385310
2,19,0,LG전자는 국내 최대 쇼핑 행사 ‘2020 코리아세일페스타’에 맞춰 11월 1일부터...,02100101.20201101125903001.txt,S-소비자,"{'S-기업(공급망)동반성장/상생': 0.007591910869623802, 'G-...",0.393870
3,20,0,SK는 그룹 8개 관계사가 한국 최초로 ‘RE100’에 가입한다고 1일 밝혔다.\n...,02100101.20201101130118001.txt,G-지배구조,"{'S-기업(공급망)동반성장/상생': 0.00795168713418085, 'G-지...",0.316108
4,20,1,SK 8개사가 신청서를 제출하면 더 클라이밋 그룹의 검토를 거친 후 가입이 최종 확...,02100101.20201101130118001.txt,S-소비자,"{'S-기업(공급망)동반성장/상생': 0.003973197078848794, 'G-...",0.269355


## Check and label predictions

In [3]:
polarity_preds_df = eKonf.load_data("esg_polarities_20220905-preds.parquet", data_dir)
category_preds_df = eKonf.load_data("esg_categories_20220905-preds.parquet", data_dir)
code_info_available = eKonf.load_data("econ_news_code_info_available_20220905.parquet", data_dir)

In [4]:
codes = ['035720', '051910', '207940', '009830']
selected = code_info_available[code_info_available['codes'].map(lambda x: x in codes)]
selected

,filename,codes,len,num_codes
21,02100101.20200102040158002.txt,035720,40,2
36,02100101.20200102080234001.txt,051910,96,10
41,02100101.20200102080234001.txt,035720,96,10
73,02100101.20200103000814001.txt,207940,47,3
151,02100101.20200108092412001.txt,207940,40,2
...,...,...,...,...
198534,02100851.20220527101334001.txt,051910,40,2
198538,02100851.20220527154429001.txt,207940,61,5
198592,02100851.20220531091955001.txt,051910,47,3
198596,02100851.20220531112630001.txt,009830,40,2


In [19]:
selected_category_preds = category_preds_df.merge(selected, on='filename')
selected_polarity_preds = polarity_preds_df.merge(selected, on='filename')

In [20]:
selected_category_preds = selected_category_preds.merge(
    selected_polarity_preds.rename(
        columns={"pred_labels": "polarity_preds"}
    )[['id', 'chunk_id', 'codes', 'polarity_preds']],
    on = ['id', 'chunk_id', 'codes']
)

In [26]:
selected_category_preds = selected_category_preds[selected_category_preds.polarity_preds != 'Neutral']
selected_polarity_preds = selected_polarity_preds[selected_polarity_preds.pred_labels != 'Neutral']

In [27]:
selected_category_preds.tail()

,id,chunk_id,text,filename,pred_labels,raw_preds,pred_probs,codes,len,num_codes,polarity_preds
46470,1090194,0,KB금융지주 신임 사외이사 후보인 최재홍 강릉원주대 멀티미디어공학과 교수. [사진=...,02100851.20220224155144001.txt,G-지배구조,"{'E-기후변화': 0.0013807213841885606, 'E-환경영향': 0....",0.638696,035720,40,2,Negative
46471,1090194,1,최 후보는 3단계로 운영되는 KB금융지주 '사추위'의 엄격한 사외이사 후보 추천 프...,02100851.20220224155144001.txt,G-지배구조,"{'E-기후변화': 0.0010913496344655172, 'E-환경영향': 0....",0.472998,035720,40,2,Positive
46475,1090506,0,"왼쪽 위부터 시계방향으로 KB금융, 신한금융, 하나금융, 우리금융. [사진=각 사]...",02100851.20220228050135001.txt,G-지배구조,"{'E-기후변화': 0.001021931625316696, 'E-환경영향': 0.0...",0.462396,035720,40,2,Positive
46477,1090506,2,우리금융은 3월 정기 주총에서 ESG 전문 여성 변호사를 사외이사로 영입할 예정이다...,02100851.20220228050135001.txt,G-지배구조,"{'E-기후변화': 0.0009028980666769455, 'E-환경영향': 0....",0.425499,035720,40,2,Positive
46478,1090506,3,금융지주 가운데 가장 많은 사외이사(12명)를 보유한 신한금융의 경우 대부분이 경제...,02100851.20220228050135001.txt,G-지배구조,"{'E-기후변화': 0.00200496373839086, 'E-환경영향': 0.00...",0.944045,035720,40,2,Positive


In [7]:
rb_cfg = eKonf.compose('model/rubrix')
rb_cfg.auto.init = True
rb = eKonf.instantiate(rb_cfg)
rb.get_workspace()

INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env
INFO:ekorpkit.base:setting environment variable CACHED_PATH_CACHE_ROOT to /workspace/.cache/cached_path
INFO:ekorpkit.base:setting environment variable KMP_DUPLICATE_LIB_OK to TRUE
INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env
INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env
INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env


'esgml'

In [30]:
meta_columns = ['id', 'chunk_id', 'filename', 'codes', 'num_codes', 'polarity_preds']
category_records = rb.create_records_from_cv_preds(
    selected_category_preds,
    meta_columns=meta_columns,
    prediction_agent="ekonelectra-base"
)
category_records[0]

TextClassificationRecord(text='지난 10월 25일 故 이건희 삼성그룹 회장이 별세하면서 삼성그룹주 주가가 요동치고 있다. 천문학적인 상속세 마련을 위해 삼성그룹 지배구조에 변화가 이뤄질 수 있다는 관측 때문이다.\n故 이건희 회장이 보유한 삼성전자, 삼성SDS, 삼성물산, 삼성생명 등 주식 가치는 18조원이 넘는 것으로 알려졌다. 부인 홍라희 전 리움미술관 관장과 이재용 삼성그룹 부회장 등 자녀들이 내야 하는 상속세만 10조원이 넘을 것으로 추정된다. 액수가 워낙 크다 보니 5년간 분할 납부하는 연부연납제도를 활용한다고 해도 상속세 신고·납부 기한인 2021년 4월까지 내야 할 첫해 상속세만 1조8000억원이 넘는다.\n결국 상속세 납부를 위해 총수 일가가 보유한 일부 삼성 계열사 지분을 처분하는 것이 불가피할 전망이다. 여기에 재원 마련을 위한 배당 확대 가능성과 이른바 ‘삼성생명법’으로 불리는 보험업법 개정안까지 얽혀 있어 계열사마다 복잡한 셈법이 작용할 것으로 예상된다. 시장에서는 벌써부터 이재용 부회장의 지분이 높은 계열사를 중심으로 주가가 강세를 나타내는 움직임을 보이고 있다. 삼성그룹의 지배구조 개편 시나리오별 주가 향방을 점쳐본다.\n▶‘이재용 체제’ 강화 움직임\n▷전면적인 지배구조 개편 가능성은 희박\n먼저 삼성 총수 일가가 납부해야 할 상속세를 들여다볼 필요가 있다. 故 이건희 회장의 지분 상속 문제는 총수 일가의 지배구조는 물론 삼성그룹 사업구조와도 밀접하게 맞물려 있다.', inputs={'text': '지난 10월 25일 故 이건희 삼성그룹 회장이 별세하면서 삼성그룹주 주가가 요동치고 있다. 천문학적인 상속세 마련을 위해 삼성그룹 지배구조에 변화가 이뤄질 수 있다는 관측 때문이다.\n故 이건희 회장이 보유한 삼성전자, 삼성SDS, 삼성물산, 삼성생명 등 주식 가치는 18조원이 넘는 것으로 알려졌다. 부인 홍라희 전 리움미술관 관장과 이재용 삼성그룹 부회장 등 자녀들이 내야 하는 상속세만 10조원이 넘을 것으로 추

In [31]:
meta_columns = ['id', 'chunk_id', 'filename', 'codes', 'num_codes']
polarity_records = rb.create_records_from_cv_preds(
    selected_polarity_preds,
    meta_columns=meta_columns,
    prediction_agent="ekonelectra-base"
)
polarity_records[0]

TextClassificationRecord(text='지난 10월 25일 故 이건희 삼성그룹 회장이 별세하면서 삼성그룹주 주가가 요동치고 있다. 천문학적인 상속세 마련을 위해 삼성그룹 지배구조에 변화가 이뤄질 수 있다는 관측 때문이다.\n故 이건희 회장이 보유한 삼성전자, 삼성SDS, 삼성물산, 삼성생명 등 주식 가치는 18조원이 넘는 것으로 알려졌다. 부인 홍라희 전 리움미술관 관장과 이재용 삼성그룹 부회장 등 자녀들이 내야 하는 상속세만 10조원이 넘을 것으로 추정된다. 액수가 워낙 크다 보니 5년간 분할 납부하는 연부연납제도를 활용한다고 해도 상속세 신고·납부 기한인 2021년 4월까지 내야 할 첫해 상속세만 1조8000억원이 넘는다.\n결국 상속세 납부를 위해 총수 일가가 보유한 일부 삼성 계열사 지분을 처분하는 것이 불가피할 전망이다. 여기에 재원 마련을 위한 배당 확대 가능성과 이른바 ‘삼성생명법’으로 불리는 보험업법 개정안까지 얽혀 있어 계열사마다 복잡한 셈법이 작용할 것으로 예상된다. 시장에서는 벌써부터 이재용 부회장의 지분이 높은 계열사를 중심으로 주가가 강세를 나타내는 움직임을 보이고 있다. 삼성그룹의 지배구조 개편 시나리오별 주가 향방을 점쳐본다.\n▶‘이재용 체제’ 강화 움직임\n▷전면적인 지배구조 개편 가능성은 희박\n먼저 삼성 총수 일가가 납부해야 할 상속세를 들여다볼 필요가 있다. 故 이건희 회장의 지분 상속 문제는 총수 일가의 지배구조는 물론 삼성그룹 사업구조와도 밀접하게 맞물려 있다.', inputs={'text': '지난 10월 25일 故 이건희 삼성그룹 회장이 별세하면서 삼성그룹주 주가가 요동치고 있다. 천문학적인 상속세 마련을 위해 삼성그룹 지배구조에 변화가 이뤄질 수 있다는 관측 때문이다.\n故 이건희 회장이 보유한 삼성전자, 삼성SDS, 삼성물산, 삼성생명 등 주식 가치는 18조원이 넘는 것으로 알려졌다. 부인 홍라희 전 리움미술관 관장과 이재용 삼성그룹 부회장 등 자녀들이 내야 하는 상속세만 10조원이 넘을 것으로 추

In [34]:
rb.log(category_records, "esg_category_prediction_check")
rb.log(polarity_records, "esg_polarity_prediction_check")

  0%|          | 0/15341 [00:00<?, ?it/s]

15341 records logged to http://ekorpkit-book:6900/datasets/esgml/esg_category_prediction_check


  0%|          | 0/13809 [00:00<?, ?it/s]

13809 records logged to http://ekorpkit-book:6900/datasets/esgml/esg_polarity_prediction_check


In [33]:
# rb.delete("esg_category_prediction_check")
# rb.delete("esg_category_prediction_check")
